# Input Features as Output Objectives

This notebook demonstrates how to put objectives on input features or a combination of input features. Possible usecases are favoring lower or higher amounts of an ingredient or to take into account a known (linear) cost function.

## Imports

In [ ]:
import bofire.strategies.api as strategies
import bofire.surrogates.api as surrogates
from bofire.benchmarks.api import Himmelblau
from bofire.data_models.features.api import ContinuousOutput
from bofire.data_models.objectives.api import MaximizeSigmoidObjective
from bofire.data_models.strategies.api import SoboStrategy
from bofire.data_models.surrogates.api import (
    BotorchSurrogates,
    LinearDeterministicSurrogate,
)

## Setup an Example

We use Himmelblau as example with an additional objective on `x_2` which pushes it to be larger 3 during the optimization.

In [ ]:
bench = Himmelblau()
experiments = bench.f(bench.domain.inputs.sample(10), return_complete=True)

domain = bench.domain
domain.outputs.features.append(
    ContinuousOutput(key="y_x2", objective=MaximizeSigmoidObjective(tp=3, steepness=10))
)

experiments["y_x2"] = experiments.x_2

The `LinearDeterministicSurrogate` can be used to model that `y_x2 = x_2`.

In [ ]:
surrogate_data = LinearDeterministicSurrogate(
    inputs=domain.inputs.get_by_keys(["x_2"]),
    outputs=domain.outputs.get_by_keys(["y_x2"]),
    coefficients={"x_2": 1},
    intercept=0,
)
surrogate = surrogates.map(surrogate_data)
surrogate.predict(experiments[domain.inputs.get_keys()].copy())

,y_x2_pred,y_x2_sd
0,-2.869398,0.0
1,-4.738913,0.0
2,-4.631332,0.0
3,-0.431255,0.0
4,4.296524,0.0
5,-2.488800,0.0
6,4.985624,0.0
7,-4.761821,0.0
8,-3.709402,0.0
9,4.985643,0.0


Next we setup a `SoboStrategy` using the custom surrogate for output `y_x2` and ask for a candidate. Note that the surrogate specs for output `y` is automatically generated and defaulted to be a `SingleTaskGPSurrogate`.

In [ ]:
strategy_data = SoboStrategy(
    domain=domain, surrogate_specs=BotorchSurrogates(surrogates=[surrogate_data])
)
strategy = strategies.map(strategy_data)
strategy.tell(experiments)
strategy.ask(1)

/Users/j30607/sandbox/botorch/botorch/models/ensemble.py:69: RuntimeWarning: Could not update `train_inputs` with transformed inputs since AffineDeterministicModel does not have a `train_inputs` attribute. Make sure that the `input_transform` is applied to both the train inputs and test inputs.
  self.eval()
/opt/homebrew/Caskroom/miniforge/base/envs/bofire-2/lib/python3.10/site-packages/torch/nn/modules/module.py:2417: RuntimeWarning: Could not update `train_inputs` with transformed inputs since AffineDeterministicModel does not have a `train_inputs` attribute. Make sure that the `input_transform` is applied to both the train inputs and test inputs.
  return self.train(False)
/Users/j30607/sandbox/botorch/botorch/models/ensemble.py:69: RuntimeWarning: Could not update `train_inputs` with transformed inputs since AffineDeterministicModel does not have a `train_inputs` attribute. Make sure that the `input_transform` is applied to both the train inputs and test inputs.
  self.eval()
/opt

,x_1,x_2,y_pred,y_x2_pred,y_sd,y_x2_sd,y_des,y_x2_des
0,-6.0,3.515752,177.233702,3.515752,126.032483,0.0,-177.233702,0.994277
